In [34]:
import torch.nn.functional as F
from torch import nn
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle,gzip,math,torch,matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
import pandas as pd
pd.DataFrame.isna = lambda x : x.isnull()

def get_data():
    mnist_url='http://deeplearning.net/data/mnist/mnist.pkl'
    path = datasets.download_data(mnist_url,ext='.gz')
    with gzip.open(path,'rb') as f:
        ((x_train, y_train), (x_valid,y_valid), _) = pickle.load(f,encoding='latin-1')
    #return map(tensor,(x_train,y_train,x_valid,y_valid))
    return x_train,y_train,x_valid,y_valid
def normalize(x,m,s):
    return (x-m)/s
mpl.rcParams['image.cmap'] = 'gray'

In [14]:
x_train,y_train,x_valid,y_valid = get_data()
m,s = x_train.mean(),x_train.std()
x_train = normalize(x_train,m,s)
x_valid = normalize(x_valid,m,s)

In [20]:
import pandas as pd
x_train = pd.DataFrame(x_train)
x_train['clase'] = y_train

x_valid = pd.DataFrame(x_valid)
x_valid['clase'] = y_valid
df = x_train.append(x_valid).reset_index(drop=True)
ids = [x_train.shape[0]+i for i in range(x_valid.shape[0])]

In [35]:
from fastai.tabular import *

In [235]:
data = TabularDataBunch.from_df(df=df,valid_idx=ids,path='.',dep_var='clase',bs=1024,cont_names=[i for i in range(784)])

In [248]:
data.show_batch(rows=5)

In [237]:
learn = tabular_learner(data,layers=[10,10,5],metrics=accuracy)

In [242]:
class ModelSaver(Callback):
    def __init__(self,fname,learn):
        self.best_accuracy=0
        self.fname=fname
        self.model=learn  
        with open(f'{self.fname}_stats.csv','w') as f:
            f.write('freeze,lr,train_loss,valid_loss,accuracy'+"\n")
        self.best_score = 0
        
    def on_epoch_end(self,last_metrics='last_metrics',smooth_loss='smooth_loss', **kwargs:Any):        
        line = str('Freeze'if self.freeze else 'Unfreeze')
        line += ","+str(self.max_lr)
        line += ","+str(round(smooth_loss.item(),4))
        line += ","+ str(round(last_metrics[0].item(),4))
        line += ","+ str(round(last_metrics[1].item(),4))        
        with open(f'{self.fname}_stats.csv','a') as f:
            f.write(line+"\n")
        
        if self.best_score<round(last_metrics[1].item(),4):
            self.best_score = round(last_metrics[1].item(),4)
            self.model.save(f"{self.fname}")
    
    def read(self):
        return pd.read_csv(f"{self.fname}_stats.csv")
        
    def fit(self,n,lr,freeze=False,**kwargs):
        self.max_lr=lr
        self.freeze=freeze
        if freeze:
            self.model.freeze()
        else:
            self.model.unfreeze()
        self.model.fit_one_cycle(n,lr,callbacks=[self],**kwargs)

In [244]:
model = ModelSaver('mati',learn)

In [246]:
model.fit(4,1e-3,moms=[0.3,.4])
model.fit(4,1e-5,moms=[0.3,.4])
model.fit(4,1e-3,moms=[0.3,.4])

epoch,train_loss,valid_loss,accuracy,time
0,1.350181,1.310459,0.750600,00:03
1,1.288122,1.214222,0.780900,00:03
2,1.215214,1.145665,0.790000,00:03
3,1.167688,1.134076,0.791800,00:02


epoch,train_loss,valid_loss,accuracy,time
0,1.138010,1.132172,0.792500,00:03
1,1.136868,1.131109,0.791800,00:02
2,1.137034,1.129260,0.791800,00:02
3,1.135239,1.133663,0.792500,00:02


epoch,train_loss,valid_loss,accuracy,time
0,1.123252,1.092348,0.793000,00:03
1,1.060517,0.987063,0.803200,00:03
2,0.991278,0.932353,0.806500,00:03
3,0.948642,0.916681,0.807500,00:03


In [247]:
model.read()

,freeze,lr,train_loss,valid_loss,accuracy
0,Unfreeze,0.00100,1.6046,1.5507,0.6308
1,Unfreeze,0.00100,1.5297,1.4335,0.6725
2,Unfreeze,0.00100,1.4503,1.3722,0.7308
3,Unfreeze,0.00100,1.3999,1.3606,0.7417
4,Unfreeze,0.00001,1.3669,1.3603,0.7400
5,Unfreeze,0.00001,1.3663,1.3595,0.7411
6,Unfreeze,0.00001,1.3652,1.3584,0.7418
7,Unfreeze,0.00001,1.3647,1.3600,0.7422
8,Unfreeze,0.00100,1.3502,1.3105,0.7506
9,Unfreeze,0.00100,1.2881,1.2142,0.7809
